In [9]:
import numpy as np

In [10]:
def MinMaxNormalized(x,type,ymin=0,ymax=1):
    """
    实现正向或负向指标MinMax归一化，返回归一化后的数据矩阵；
    x：为原始数据矩阵, 一行代表一个样本, 每列对应一个指标；
    type：设定正向指标1,负向指标2；
    ymin,ymax：归一化的区间端点，即归一化时将数据缩放到(ymin,ymax)的范围内，默认应设置为[0,1]；
    """
    n, m = x.shape
    y = np.zeros((n, m))
    xmin = np.min(x,axis=0)
    xmax = np.max(x,axis=0)
    if type == 1:
        for j in range(m):
            y[:,j] = (ymax-ymin)*(x[:,j]-xmin[j]) / (xmax[j]-xmin[j])+ymin
    elif type == 2:
        for j in range(m):
            y[:,j] = (ymax-ymin)*(xmax[j]-x[:,j]) / (xmax[j]-xmin[j])+ymin
    return y

In [11]:
def Entropy_Weight_Method(x,ind,ymin=0.0001,ymax=0.9999):
    """熵权法（EWM）
    输入变量
    x: 原始数据矩阵, 一行代表一个样本, 每列对应一个指标；
    ind: 指示向量，指示各列正向指标还是负向指标，1表示正向指标，2表示负向指标；
    输出变量
    Y:归一化后的数据
    s:综合加权评分
    w: 各指标权重；
    """
    # n个样本, m个指标
    n,m = x.shape
    # 数据的归一化处理
    # 归一化结果
    Y = np.zeros((n, m))
    for i in range(m):
        if ind[i] == 1: #正向指标归一化
            Y[:,i] = MinMaxNormalized(x[:,i].reshape(-1,1),1,ymin,ymax).flatten()
        elif ind[i] == 2: #负向指标归一化
            Y[:,i] = MinMaxNormalized(x[:,i].reshape(-1,1),2,ymin,ymax).flatten()
    # 计算第m项指标下第m个样本值占该指标的比重：比重P(i,j)
    P = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            P[i,j] = Y[i,j] / np.sum(Y[:,j])
    # 第j个指标的熵值e(j)
    e = np.zeros((1,m))
    # 其中k = 1/ln(n)
    k= 1/np.log10(n)
    for j in range(m):
        e[0,j] = -k * np.sum(P[:,j] * np.log10(P[:,j]))
    # 计算信息熵冗余度
    d = np.ones_like(e)-e
    # 计算各项指标的权重
    w = d / np.sum(d) 
    # 计算该样本的综合加权评分
    s = np.sum(w * Y,axis=1).reshape(-1,1)
    return Y,s, w